# Modelado Analítico: Star Schema

Este notebook demuestra cómo crear y trabajar con un modelo analítico tipo Star Schema.

**Referencia:** [Modelado Analítico](../modelado/modelado-analitico.md)

**Objetivos:**
- Crear tablas de hechos y dimensiones
- Visualizar estructura del modelo
- Consultar datos usando el modelo

## 1. Importar librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Librerías importadas")

## 2. Crear tablas de dimensiones

In [ ]:
# Dimension: Tiempo
fechas = pd.date_range('2024-01-01', '2024-12-31', freq='D')
dim_tiempo = pd.DataFrame({
    'fecha_id': range(1, len(fechas) + 1),
    'fecha': fechas,
    'año': fechas.year,
    'mes': fechas.month,
    'dia_semana': fechas.dayofweek,
    'trimestre': fechas.quarter,
    'es_fin_semana': fechas.dayofweek.isin([5, 6])
})

print(f"✅ Dimension Tiempo creada: {len(dim_tiempo)} registros")
dim_tiempo.head()

In [ ]:
# Dimension: Producto
productos = ['Producto A', 'Producto B', 'Producto C', 'Producto D', 'Producto E']
categorias = ['Electrónica', 'Ropa', 'Hogar', 'Deportes', 'Libros']

dim_producto = pd.DataFrame({
    'producto_id': range(1, len(productos) + 1),
    'nombre': productos,
    'categoria': categorias,
    'precio_base': [100, 50, 75, 120, 25]
})

print(f"✅ Dimension Producto creada: {len(dim_producto)} registros")
dim_producto

In [ ]:
# Dimension: Cliente
clientes = [f'Cliente {i}' for i in range(1, 11)]
regiones = np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], 10)

dim_cliente = pd.DataFrame({
    'cliente_id': range(1, 11),
    'nombre': clientes,
    'region': regiones,
    'tipo': np.random.choice(['Premium', 'Regular'], 10)
})

print(f"✅ Dimension Cliente creada: {len(dim_cliente)} registros")
dim_cliente.head()

## 3. Crear tabla de hechos (Fact Table)

In [ ]:
# Tabla de hechos: Ventas
np.random.seed(42)
n_ventas = 1000

fact_ventas = pd.DataFrame({
    'venta_id': range(1, n_ventas + 1),
    'fecha_id': np.random.choice(dim_tiempo['fecha_id'], n_ventas),
    'producto_id': np.random.choice(dim_producto['producto_id'], n_ventas),
    'cliente_id': np.random.choice(dim_cliente['cliente_id'], n_ventas),
    'cantidad': np.random.randint(1, 10, n_ventas),
    'precio_unitario': np.random.uniform(50, 150, n_ventas),
    'descuento': np.random.uniform(0, 0.2, n_ventas)
})

# Calcular total
fact_ventas['total'] = fact_ventas['cantidad'] * fact_ventas['precio_unitario'] * (1 - fact_ventas['descuento'])

print(f"✅ Fact Table Ventas creada: {len(fact_ventas)} registros")
fact_ventas.head()

## 4. Visualizar estructura del modelo

In [ ]:
print("=" * 60)
print("ESTRUCTURA STAR SCHEMA")
print("=" * 60)
print("\n        Dimension: Tiempo")
print("              |")
print("              |")
print("    Dimension: Producto --- Fact: Ventas --- Dimension: Cliente")
print("              |")
print("              |")
print("        Dimension: Cliente")
print("\n" + "=" * 60)
print(f"\nFact Table: {len(fact_ventas)} registros")
print(f"Dimension Tiempo: {len(dim_tiempo)} registros")
print(f"Dimension Producto: {len(dim_producto)} registros")
print(f"Dimension Cliente: {len(dim_cliente)} registros")

## 5. Consultar datos usando el modelo

In [ ]:
# JOIN para análisis completo
ventas_completo = fact_ventas.merge(
    dim_tiempo[['fecha_id', 'fecha', 'mes', 'trimestre']], 
    on='fecha_id'
).merge(
    dim_producto[['producto_id', 'nombre', 'categoria']], 
    on='producto_id'
).merge(
    dim_cliente[['cliente_id', 'nombre', 'region', 'tipo']], 
    on='cliente_id'
)

print("✅ Datos combinados (simulando JOIN SQL)")
ventas_completo.head()

## 6. Análisis usando el modelo

In [ ]:
# Ventas por categoría
ventas_categoria = ventas_completo.groupby('categoria')['total'].agg(['sum', 'mean', 'count']).round(2)
ventas_categoria.columns = ['Total', 'Promedio', 'Cantidad']
print("=== VENTAS POR CATEGORÍA ===")
print(ventas_categoria)

In [ ]:
# Ventas por trimestre
ventas_trimestre = ventas_completo.groupby('trimestre')['total'].sum()
print("\n=== VENTAS POR TRIMESTRE ===")
print(ventas_trimestre)

# Visualizar
plt.figure(figsize=(10, 6))
ventas_trimestre.plot(kind='bar', color='steelblue')
plt.title('Ventas por Trimestre')
plt.xlabel('Trimestre')
plt.ylabel('Total Ventas (€)')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Ventas por región y tipo de cliente
ventas_region_tipo = ventas_completo.groupby(['region', 'tipo'])['total'].sum().unstack(fill_value=0)
print("\n=== VENTAS POR REGIÓN Y TIPO ===")
print(ventas_region_tipo)

# Visualizar
plt.figure(figsize=(10, 6))
ventas_region_tipo.plot(kind='bar', stacked=False)
plt.title('Ventas por Región y Tipo de Cliente')
plt.xlabel('Región')
plt.ylabel('Total Ventas (€)')
plt.legend(title='Tipo Cliente')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## 7. Ventajas del Star Schema

In [ ]:
print("=" * 60)
print("VENTAJAS DEL STAR SCHEMA")
print("=" * 60)
print("\n✅ Fácil de entender para usuarios de negocio")
print("✅ Consultas más rápidas (menos JOINs complejos)")
print("✅ Optimizado para lectura (analytics)")
print("✅ Escalable (dimensiones pequeñas, facts grandes)")
print("✅ Flexible para agregaciones")
print("=" * 60)

---

**Próximo paso:** Lee [Modelado Analítico](../modelado/modelado-analitico.md) para más detalles sobre diseño de modelos.